In [52]:
import pandas as pd
import numpy as np
import calendar

In [39]:
# Load companies.json, jobs.json and locations.csv into DataFrames
companies_data = pd.read_json('data/companies.json')
jobs_data = pd.read_json('data/jobs.json')
locations_data = pd.read_csv('data/locations.csv')

#Let's have a look at the data in these data frames:
print(companies_data.head())
print(companies_data.info())
print(jobs_data.head())
print(jobs_data.info())
print(locations_data.head())
print(locations_data.info())

   Company ID               Company Name  Establishment Date  \
0           0                  Acme Inc.       1577750400000   
1           1                 Best Corp.       1616284800000   
2           2  Bright Future Enterprises       1605312000000   
3           3                 Delta Inc.       1604620800000   
4           4           Echo Enterprises       1651708800000   

   Number of Employees  
0                  404  
1                  229  
2                  222  
3                  662  
4                  255  
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Company ID           10 non-null     int64 
 1   Company Name         10 non-null     object
 2   Establishment Date   10 non-null     int64 
 3   Number of Employees  10 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 400.0+ bytes
None
          

In [40]:
# ********************************* Data quality checks *********************************

# The Establishment Date is originally a Unix timestamp, and so let's convert it to a regular timestamp
companies_data['Establishment Date'] = pd.to_datetime(companies_data['Establishment Date'], unit='ms', origin='unix')
print("Establishment Date has been converted to a regular timestamp.\n")

# Let's make sure that the values of the "state" column are all correct
distinct_states = jobs_data['state'].value_counts().index.tolist()
print(distinct_states)

# Some of th state values seem to be wrong, so let's correct them:
jobs_data['state'] = jobs_data['state'].replace({'cancelled': 'canceled', 'osted': 'posted'})

# Let's check the changes
print(jobs_data['state'].unique())
print("The state column in jobs_data has been corrected.\n")

# Let's also check for rows where the "zip_code" column is null
null_zip_code_rows = jobs_data['zip'].isnull().sum()
print("Total rows with null zip in jobs dataframe:", null_zip_code_rows)

#Let's define a funtion to check if there are any duplicated rows
def check_duplicates(dataframe):
    # Check for duplicates and return the duplicated rows if any are found
    duplicates = dataframe[dataframe.duplicated(keep=False)]
    if not duplicates.empty:
        print("Duplicates found:")
        print(duplicates)
    else:
        print("No duplicates found.")
    
#Now let's use this funtion to detecte duplicated rows:
print("\nChecking duplicated rows for jobs_data...")
check_duplicates(jobs_data)

print("\nChecking duplicated rows for locations_data...")
check_duplicates(locations_data)

print("\nChecking duplicated rows for companies_data...")
check_duplicates(companies_data)

#Duplicated rows have been found in jobs_data, and so let's drop the duplicated rows:
jobs_data = jobs_data.drop_duplicates(keep='first')
print("\nDuplicated rows have been dropped in the dataframe jobs_data.")

Establishment Date has been converted to a regular timestamp.

['cancelled', 'expired', 'posted', 'osted', 'canceled']
['expired' 'canceled' 'posted']
The state column in jobs_data has been corrected.

Total rows with null zip in jobs dataframe: 0

Checking duplicated rows for jobs_data...
Duplicates found:
              posted_at  id     state    zip   price  company_id
2   2021-07-21 15:29:00   3   expired  35786  335.85           8
96  2022-12-29 16:05:00  97  canceled  78956   11.83           1
100 2022-12-29 16:05:00  97  canceled  78956   11.83           1
101 2021-07-21 15:29:00   3   expired  35786  335.85           8

Checking duplicated rows for locations_data...
No duplicates found.

Checking duplicated rows for companies_data...
No duplicates found.

Duplicated rows have been dropped in the dataframe jobs_data.


In [41]:
#1. What location has the most jobs that are either posted or expired?
# Join the "jobs" and "locations" DataFrames based on the condition zip=zip_code
joined_data = jobs_data.merge(locations_data, left_on='zip', right_on='zip_code', how='inner')
#print(joined_data)

# Filter for jobs that are either "posted" or "expired"
joined_data = joined_data[joined_data['state'].isin(['posted', 'expired'])]
#print(joined_data)

# Group by location and count the number of jobs in each location
location_job_counts = joined_data['location'].value_counts().reset_index()
location_job_counts.columns = ['Location', 'Job Count']

# Find the location with the highest job count
max_job_location = location_job_counts.loc[location_job_counts['Job Count'].idxmax()]

print("Location with the most jobs (posted or expired):")
print(max_job_location)

Location with the most jobs (posted or expired):
Location     Tarth
Job Count       15
Name: 0, dtype: object


In [42]:
#2. What month had the most cancelled jobs? (Just the month without the year)

jobs_data['posted_at_month'] = jobs_data['posted_at'].dt.month
canceled_jobs = jobs_data[jobs_data['state'] == 'canceled']

# Group by month and count canceled jobs
canceled_jobs_by_month = canceled_jobs.groupby('posted_at_month')['state'].count()

# Get the month with the highest count
most_canceled_month = canceled_jobs_by_month.idxmax()

print("Month with the most canceled jobs:", calendar.month_name[most_canceled_month])

Month with the most canceled jobs: June


In [43]:
#2. What month had the most cancelled jobs? (Month and year)
jobs_data['posted_at_month'] = jobs_data['posted_at'].dt.month
jobs_data['posted_at_year'] = jobs_data['posted_at'].dt.year
canceled_jobs = jobs_data[jobs_data['state'] == 'canceled']

# Group by month and count canceled jobs
canceled_jobs_by_month_year = canceled_jobs.groupby(['posted_at_month', 'posted_at_year'])['state'].count()

# Get the month with the highest count
most_canceled_month_year = canceled_jobs_by_month_year.idxmax()

print("Month and year with the most canceled jobs:", most_canceled_month_year)

Month and year with the most canceled jobs: (6, 2021)


In [44]:
#3. Which company has the highest ratio of posted jobs to employee count?

# We already know the employee count from the data provided, let's count the jobs count per company
posted_jobs = jobs_data[jobs_data['state'] == 'posted']
posted_jobs = posted_jobs.groupby('company_id').size().reset_index(name='jobs_count')
posted_jobs.set_index('company_id', inplace=True)

posted_jobs_employee = posted_jobs.merge(companies_data, left_on='company_id', right_on='Company ID', how='inner')
posted_jobs_employee['ratio_posted_jobs_employee'] = posted_jobs_employee['jobs_count'] / posted_jobs_employee['Number of Employees']

sorted_posted_jobs_employee = posted_jobs_employee.sort_values(by='ratio_posted_jobs_employee', ascending=False)

# Get the company with the highest ratio
highest_ratio_company = sorted_posted_jobs_employee.iloc[0]

# Print the company name and the highest ratio
print("Company with the highest ratio:")
print("Company Name:", highest_ratio_company['Company Name'])
print("Ratio:", highest_ratio_company['ratio_posted_jobs_employee'])


Company with the highest ratio:
Company Name: Best Corp.
Ratio: 0.017467248908296942
